In [1]:
import pandas as pd
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import wandb
import timm

In [2]:
import time
import matplotlib.pyplot as plt
import copy
import sklearn.metrics as metrics
from tqdm import tqdm

In [1]:
import os



# !kaggle competitions download -c vk-made-sports-image-classification

In [4]:
competition_name = "vk-made-sports-image-classification"

In [5]:
# !kaggle competitions submit -c {competition_name} -f {svd_sol_filepath} -m 'my first svd submission'

In [6]:
# !unzip vk-made-sports-image-classification.zip

In [7]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [8]:
df_train["label"].value_counts()

basketball               2519
hockey                   2151
football                 2150
running                  1991
rugby                    1967
skating                  1957
alpinism                 1858
volleyball               1856
boxing                   1801
boating                  1753
tennis                   1735
horseback_riding         1568
handball                 1509
fencing                  1421
surfing                  1420
gymnastics               1415
golf                     1393
arm_wrestling            1392
ski_race                 1335
greco-Roman_wrestling    1298
swimming                 1283
sambo                    1264
velo                     1216
sailing                  1181
badminton                1140
taekwondo                1110
pole_vault               1047
archery                   982
water_polo                850
javelin-throwing          811
Name: label, dtype: int64

In [9]:
df_test

,image_id
0,00fd3c23-193c-480a-aef9-bb438d50d79e.jpeg
1,ef5473b9-a558-4f38-acd0-be4ecfde5a23.jpeg
2,ba00f9b4-7cbf-4110-91ea-ed41f5cb4ee4.jpeg
3,f1bdf877-4379-4e00-b5b8-1e90bdcbda76.jpeg
4,4c96ff83-07e3-45a4-934f-a92ebf49b299.jpeg
...,...
19441,01ec2a50-db12-4b98-94da-1dd9372650cf.jpeg
19442,1596a0c0-5772-4309-8781-47e15dcdd5d5.jpeg
19443,65610b2c-c137-468a-85a6-a3ec2ed9e87f.jpeg
19444,5630e635-fce1-4a33-8748-b7416d16fffc.jpeg


In [10]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()
label_enc.fit(df_train["label"])
classes = label_enc.transform(df_train["label"])
df_train["classes"] = classes

In [14]:
df_train

,image_id,label,classes
0,46514481-2d8b-4d49-8991-012e1bfd34f6.jpeg,swimming,24
1,ec66e513-adac-4a30-b6a9-3d647ee6e46b.jpeg,greco-Roman_wrestling,10
2,4d60732e-d680-4bfd-9067-70ff8137f537.jpeg,running,18
3,93327011-8e3d-4f0d-849d-a26ddaf6488b.jpeg,football,8
4,b6853478-48c1-48b2-b104-74903730c831.jpeg,sailing,19
...,...,...,...
45368,b53fcada-9046-44e4-92f9-8c1a0dc60fb3.jpeg,handball,12
45369,53dd096c-3697-4e3b-99b6-e77c1129b798.jpeg,greco-Roman_wrestling,10
45370,5e26219c-4629-42fe-b037-fb6b1bf78f7f.jpeg,surfing,23
45371,c43f5525-44e4-48f6-a8ef-95ebe569bfc6.jpeg,pole_vault,16


In [15]:
df_train.iloc[0, 0]

'46514481-2d8b-4d49-8991-012e1bfd34f6.jpeg'

In [16]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((384, 384)), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])



In [17]:
class CustomDataset(Dataset):

    def __init__(self, data_frame, transform=transform, root_dir=None, is_test=False):
        self.df = data_frame
#         self.labels = self.df['classes'].to_list()
#         self.paths = self.df['audio_path'].to_list()
        self.root_dir = root_dir
        self.is_test = is_test
        self.transform = transform
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")

        img_tensor = self.transform(image)
        if self.is_test:
            return img_tensor
        label = torch.tensor(self.df.iloc[idx, 2])
        return img_tensor, label

In [18]:
trainset = CustomDataset(data_frame=df_train, transform=transform, root_dir="./train/")
testset = CustomDataset(data_frame=df_test, transform=transform, root_dir="./test/", is_test=True)

In [19]:
img, la = next(iter(trainset))

/opt/conda/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [20]:
d = df_train["classes"].value_counts().to_dict()

d1 = {}
for i in range(len(d)):
    d1[i] = 0
    
for i in range(len(d)):
    d1[i] = d[i]
    
s = len(df_train)
s

for i in range(len(d1)):
    d1[i] = s / d1[i]
    
ss = sum(d1.values())

for i in range(len(d1)):
    d1[i] = d1[i] / ss

In [21]:
weights = torch.tensor(list(d1.values()))

In [22]:
from torchvision.models import resnet50

In [23]:
# timm.list_models(pretrained=True)

In [24]:
# model = timm.create_model("convnext_small_384_in22ft1k", pretrained=True)

In [25]:
wandb.init(project="made-sports-classification", name = "big", tags = ["Big model"])

wandb: Currently logged in as: 68228322. Use `wandb login --relogin` to force relogin


In [43]:
model = timm.create_model("convnext_small_384_in22ft1k", pretrained=True)
model.head.fc = torch.nn.Linear(768, 30)

In [45]:
# model.head

In [46]:
for param in model.parameters():
    param.requires_grad = False
model.head.fc.requires_grad_(True)

Linear(in_features=768, out_features=30, bias=True)

In [93]:
def train(model, dataloaders, criterion, optimizer, num_epochs=1, scheduler=None, savename=None, print_counter=10, decay_factor=100, device=torch.device("cuda")):
    
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        torch.save(model.state_dict(), savename+"epoch_checkpoint.pt")
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        print("-" * 10)
        for phase in ["train", "val"]: ## ["train", "val"]
            if phase == "train":
                model.train()
                model.to(device)
            else:
                model.eval()

            epoch_loss = 0
            epoch_correct = 0
            for i, (inputs, labels) in enumerate(tqdm(dataloaders[phase])):
                
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase=="train"):
                    outputs = model(inputs)
                    softmax = nn.Softmax(dim=1)
                    loss = criterion(outputs, labels)
                    outputs = softmax(outputs)
                    preds = torch.argmax(outputs, dim=1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss = loss.item() * inputs.size(0)
                epoch_loss += running_loss
                running_corrects = torch.sum(preds == labels.data)
                epoch_correct += running_corrects

                if phase == "train":
                    wandb.log({"running_train_loss": running_loss, "running_train_accuracy": (torch.sum(preds == labels.data)/len(labels.data)).item() })
                else:
                    wandb.log({"running_val_loss": running_loss, "running_val_accuracy": (torch.sum(preds == labels.data)/len(labels.data)).item() })
                
                if i % 500 == 0:
                    torch.save(model.state_dict(), savename+ "Epoch_" + str(epoch + 1)+"_i_"+str(i)+".pt")
                    
                if i % print_counter == 0:
                    print("Running Loss", loss.item(), "| Running accuracy ",  (torch.sum(preds == labels.data)/len(labels.data)).item() )
                
                

            epoch_loss = epoch_loss / len(dataloaders[phase].dataset)
            epoch_acc = epoch_correct.double() / len(dataloaders[phase].dataset)
            if phase == "train":
                wandb.log({"epoch":epoch + 1, "epoch_train_loss": epoch_loss, "epoch_train_accuracy": epoch_acc})
            else:
                wandb.log({"epoch":epoch + 1, "epoch_val_loss": epoch_loss, "epoch_val_accuracy": epoch_acc})
                
            
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            if phase == "train":
                scheduler.step()
            if phase == "train":
                try:
                    optimizer.update_regularizer(decay_factor=decay_factor)
                except:
                    pass
                
            # deep copy the model
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), savename+"checkpoint.pt")
            if phase == "val":
                val_acc_history.append(epoch_acc)
            
            

        print()

    time_elapsed = time.time() - since
    print("Training complete in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60))
#     print("Best val Acc: {:4f}".format(best_acc))

    # load best model weights
    torch.save(model.state_dict(), savename+"finished.pt")
    model.load_state_dict(best_model_wts)
    
    return model, val_acc_history

In [94]:
def test(model, test_dataloader, device=None):

    model = model.eval()
    model.to(device)
    phase = "test"
    running_corrects = 0
    preds = torch.ones(1)
    correct = 0
    with torch.no_grad():
        for inputs in tqdm(test_dataloader):
            inputs = inputs.to(device)
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                softmax = nn.Softmax(dim=1)
                outputs = softmax(outputs)
                running_preds = torch.argmax(outputs, dim=1)
                preds = torch.cat((preds, running_preds.view(-1).detach().cpu()), dim=0)      
    preds = np.array(preds[1:])
    
    return preds

In [95]:
import torch.nn as nn
import numpy as np

In [92]:
df_train

,image_id,label,classes
0,46514481-2d8b-4d49-8991-012e1bfd34f6.jpeg,swimming,24
1,ec66e513-adac-4a30-b6a9-3d647ee6e46b.jpeg,greco-Roman_wrestling,10
2,4d60732e-d680-4bfd-9067-70ff8137f537.jpeg,running,18
3,93327011-8e3d-4f0d-849d-a26ddaf6488b.jpeg,football,8
4,b6853478-48c1-48b2-b104-74903730c831.jpeg,sailing,19
...,...,...,...
45368,b53fcada-9046-44e4-92f9-8c1a0dc60fb3.jpeg,handball,12
45369,53dd096c-3697-4e3b-99b6-e77c1129b798.jpeg,greco-Roman_wrestling,10
45370,5e26219c-4629-42fe-b037-fb6b1bf78f7f.jpeg,surfing,23
45371,c43f5525-44e4-48f6-a8ef-95ebe569bfc6.jpeg,pole_vault,16


In [118]:
def main(model, DF_TRAIN):
    
    lr = 1e-3
    epochs = 10
    batch_size = 32
    wandb.init(project="made-sports-classification", name = "newone", tags = ["Big model"])
    wandb.config = {
      "learning_rate": lr,
      "epochs": epochs,
      "batch_size": batch_size
    }
    train_dataset_wavs = CustomDataset(data_frame=DF_TRAIN, transform=transform, root_dir="./train/") #.iloc[0:200] #.iloc[:-2000]
    train_dataloader_wavs = DataLoader(train_dataset_wavs, batch_size=batch_size, shuffle=True, num_workers=8)

    val_dataset_wavs = CustomDataset(data_frame=DF_TRAIN.iloc[-2000:], transform=transform, root_dir="./train/")
    val_dataloader_wavs = DataLoader(val_dataset_wavs, batch_size=batch_size, shuffle=False, num_workers=8)
    dataloaders = {"train": train_dataloader_wavs, "val": val_dataloader_wavs}
    
    
    device = torch.device("cuda:2") 
    # device = torch.device("cpu") 
                                                                                           

    model = model.to(device)
#     for param in model.parameters():
#         param.requires_grad = False
#     model.head.fc.requires_grad_(True)
                                                                                           
#     optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum = 0.99)
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = 0.8)
    criterion = nn.CrossEntropyLoss(weight=weights.to(device))

    
    for_ckpts = "."
    os.makedirs(for_ckpts, exist_ok=True)
    
    
    
    model, val_acc_history = train(model, dataloaders, criterion, optimizer, num_epochs=epochs, scheduler=scheduler, device=device, savename=for_ckpts)
    
    
    return model

In [119]:
model = timm.create_model("convnext_small_384_in22ft1k", pretrained=True)
model.head.fc = torch.nn.Linear(768, 30)

for param in model.parameters():
    param.requires_grad = False
    
model.head.requires_grad_(True)

model.stages[3].requires_grad_(True)
2+2

4

In [ ]:
m1 = main(model, df_train)

running_train_accuracy,▁▆▆▅▄▇▃▅▄▇▆▆▅▆▆▄▆▆▅▆▅▆▆▄▄▅▆▇▅▆▅▆▇█▆▅▆▆▄▇
running_train_loss,█▂▃▄▄▁▄▄▅▂▄▃▃▄▂▄▄▃▃▃▄▇▃▄▄▄▃▃▃▃▄▃▂▁▂▄▄▂▅▂
running_train_accuracy,0.9375
running_train_loss,10.1144


Epoch 0/9
----------


  0%|                                                  | 0/1418 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc

Running Loss 3.5085792541503906 | Running accuracy  0.03125


  1%|▎                                        | 11/1418 [00:10<10:01,  2.34it/s]

...

 94%|████████████████████████████████████▌  | 1331/1418 [05:28<00:21,  4.09it/s]

Running Loss 0.07010895013809204 | Running accuracy  0.96875


 95%|████████████████████████████████████▉  | 1341/1418 [05:30<00:18,  4.07it/s]

Running Loss 0.13692854344844818 | Running accuracy  0.96875


 95%|█████████████████████████████████████▏ | 1351/1418 [05:33<00:16,  4.09it/s]

In [ ]:
test_dataset_wavs = CustomDataset(data_frame=df_test, transform=transform, root_dir="./test/", is_test=True)
test_dataloader_wavs = DataLoader(test_dataset_wavs, batch_size=32, shuffle=False, num_workers=8)

In [ ]:
preds = test(m1, test_dataloader_wavs, device="cuda:2")

In [ ]:
preds

In [ ]:
test_preds = label_enc.inverse_transform(preds.astype("int"))

In [ ]:
len(test_preds)

In [ ]:
df_test["label"] = test_preds

In [ ]:
df_test.to_csv("first_submission.csv", index=False)

In [ ]:
df_test

In [ ]:
svd_sol_filepath = "first_submission.csv"

In [ ]:
!kaggle competitions submit -c {competition_name} -f {svd_sol_filepath} -m '5th submission, 5e frozen less'

# References
- https://pytorch.org/tutorials/beginner/data_loading_tutorial.html